In [1]:
## Importing all the necessary libraries

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Defining our Network

In [19]:
## Defining our Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1,bias=False),    
            # no. of input channels = 1, no. of output channels = 16, kernel size = 3x3, input size = 28x28, OUtput size = 28x28 
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.05),
            nn.MaxPool2d(2, 2),                           
            # input size = 28x28, Output size = 14x14
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1,bias=False),   
            # no. of input channels = 16, no. of output channels = 32, kernel size = 3x3, input size = 14x14, OUtput size = 14x14
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.05),
            nn.MaxPool2d(4, 4),                           
            # input size = 14x14, Output size = 3x3
        )    
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 50, 3, padding=1,bias=False),   
            # no. of input channels = 32, no. of output channels = 50, kernel size = 3x3, input size = 3x3, OUtput size = 3x3
            nn.ReLU(),
            nn.BatchNorm2d(50),
            nn.Dropout(0.05),
            nn.MaxPool2d(2, 2),  
            # input size = 3x3, Output size = 1x1
        )
      
        self.fc = nn.Sequential(
            nn.Linear(50, 10, bias=False),                
            # input size = 1x1x50 = 50, Output size = 10
        )                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
      
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

## Model Summary

In [20]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
           Dropout-4           [-1, 16, 28, 28]               0
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 32, 14, 14]           4,608
              ReLU-7           [-1, 32, 14, 14]               0
       BatchNorm2d-8           [-1, 32, 14, 14]              64
           Dropout-9           [-1, 32, 14, 14]               0
        MaxPool2d-10             [-1, 32, 3, 3]               0
           Conv2d-11             [-1, 50, 3, 3]          14,400
             ReLU-12             [-1, 50, 3, 3]               0
      BatchNorm2d-13             [-1, 50, 3, 3]             100
          Dropout-14             [-1, 5

In [21]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Test and Train Functions

In [22]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

##  Training and Testing

In [23]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_printoptions(edgeitems=40)

epoch_number = 1
for epoch in range(1, 20):
    print("Epoch Number = ",epoch_number)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    epoch_number += 1

Epoch Number =  1


loss=0.062293101102113724 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.11it/s]



Test set: Average loss: 0.0492, Accuracy: 9847/10000 (98%)

Epoch Number =  2


loss=0.040206607431173325 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.78it/s]



Test set: Average loss: 0.0356, Accuracy: 9886/10000 (99%)

Epoch Number =  3


...


Test set: Average loss: 0.0278, Accuracy: 9920/10000 (99%)

Epoch Number =  18


loss=0.001436802209354937 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.98it/s]



Test set: Average loss: 0.0256, Accuracy: 9924/10000 (99%)

Epoch Number =  19


loss=0.004901103209704161 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.54it/s]



Test set: Average loss: 0.0275, Accuracy: 9928/10000 (99%)

